In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
import impyute.imputation.cs.fast_knn as knn
import impyute.imputation.cs.em as EM
import impyute.imputation.cs.mice as mice
import impyute.imputation.ts.locf as locf

In [3]:
import random as rand

from sklearn.experimental import enable_iterative_imputer 
from sklearn.impute import SimpleImputer

## Load .csv File

### Load price and volume files and repalce the 'NR' value to np.NaN

In [27]:
crop = 'brinjal'
price_df = pd.read_csv(f'../price_{crop}.csv', index_col=0, parse_dates=True, low_memory=False)
volume_df = pd.read_csv(f'../volume_{crop}.csv', index_col=0, parse_dates=True, low_memory=False)
price_df.replace(to_replace='NR', value=np.NaN, inplace=True)
volume_df.replace(to_replace='NR', value=np.NaN, inplace=True)
price_df

,Gudiwada_Andh,Kakinada_Andh,Machilipatnam_Andh,Rajahmundry_Andh,Vijayawada_Andh,Bongiagaon_Assa,Cachar_Assa,Howly_Assa,Lanka_Assa,P.O. Uparhali Guwahati_Assa,...,Bhulath (Nadala)_Punj,Cuddapah_Andh,Khambhat_Guja,Kullu(Patli Kuhal)_Hima,Allagadda_Andh,Kottur_Karn,Adilabad_Tela,Sirmour(Bagthan)_Hima,Tarori_Hary,Dharamshala_Hima
2008-01-01,NaN,NaN,NaN,650,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2008-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,750,1350,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2008-01-03,NaN,NaN,NaN,650,NaN,NaN,800,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2008-01-04,NaN,NaN,NaN,550,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2008-01-05,NaN,NaN,NaN,550,NaN,NaN,NaN,NaN,NaN,1300,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2008-01-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2008-01-07,NaN,NaN,NaN,800,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2008-01-08,NaN,NaN,NaN,900,NaN,NaN,NaN,NaN,NaN,825,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2008-01-09,NaN,NaN,NaN,800,NaN,NaN,NaN,NaN,NaN,1050,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2008-01-10,NaN,NaN,NaN,1050,NaN,NaN,NaN,NaN,550,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Fill NaN with 0.0

In [28]:
# price_df.fillna(0., inplace=True)
# price_df.astype('float32')
# volume_df.fillna(0., inplace=True)
# volume_df.astype('float32')

### Drop columns with observation's freqeuncy < 0.1

f = observation / days
Drop the columns which f < 0.1 and f_last_1000_days < 0.3

In [29]:
print(f"total length: {len(price_df.columns)}")
drop_list = []
days_n = len(price_df.index)

markets = price_df.columns

# for market in markets:
#     total_non_zero = len(list(np.where(price_df[market] != 0.0)[0]))
#     last_1000_d_non_zero = len(list(np.where(price_df[market].iloc[-1000: -1] != 0.0)[0]))
#     if (total_non_zero / days_n < 0.1) and (last_1000_d_non_zero / 1000 < 0.3):
#         drop_list.append(market)

for i, f in enumerate(price_df.isna().sum() / days_n):
    if f > 0.9:
        drop_list.append(price_df.columns[i])

# drop_list = [f for f in df.isna().sum() / days if f > 0.1]
print(f"drop length: {len(drop_list)}")

price_df_ = price_df.drop(columns=drop_list)
volume_df_ = volume_df.drop(columns=drop_list)


total length: 1332
drop length: 665


In [33]:
price_m = price_df_.to_numpy(dtype=np.float64)
volume_m = volume_df_.to_numpy(dtype=np.float64)

print(f"price_m shape: {price_m.shape}")
print(f"volume_m shape: {volume_m.shape}")


price_m shape: (4018, 667)
volume_m shape: (4018, 667)


### EM Algorithm

In [ ]:
# m_ = knn(m)
imputed_price_m = EM(price_m)
print('finish impute price matrix')
imputed_volume_m = EM(volume_m)
print('finish impute volume matrix')

In [13]:
imputed_price_em = imputed_price_m
imputed_volume_em = imputed_volume_m

Wall time: 0 ns


### MICE

In [ ]:
%%time
imputed_price_mice = mice(price_m)
print('finish impute price matrix')
imputed_volume_mice = mice(volume_m)
print('finish impute volume matrix')

### Last Observation

In [34]:
%%time
imputed_price_locf = locf(price_m)
print("finish impute price")
imputed_volume_locf = locf(volume_m)
print("finish impute volume")

finish impute price
finish impute volume
Wall time: 10.2 s


In [35]:
print(f"shape: {imputed_price_locf.shape}")
print(f"imputed_price_locf: {imputed_price_locf}")

shape: (667, 4018)
imputed_price_locf: [[ 550.  750.  800. ... 1400. 1400. 1400.]
 [ 550.  750.  800. ... 1400. 1400. 1400.]
 [ 550.  750.  800. ... 1400. 1400. 1400.]
 ...
 [ 275.  300.  400. ...  580.  485.  580.]
 [ 275.  300.  400. ...  580.  485.  580.]
 [ 275.  300.  400. ...  400.  485.  400.]]


In [36]:
price_df = pd.DataFrame(index=pd.date_range('2008-1-1', '2018-12-31'), 
                        data=imputed_price_locf.transpose(), columns=price_df_.columns)
volume_df = pd.DataFrame(index=pd.date_range('2008-1-1', '2018-12-31'), 
                         data=imputed_volume_locf.transpose(), columns=price_df_.columns)

In [9]:
# numpy array to dataframe
price_df = pd.DataFrame(index=pd.date_range('2008-1-1', '2018-12-31'), 
                        data=imputed_price_m.transpose(), columns=price_df_.columns)
volume_df = pd.DataFrame(index=pd.date_range('2008-1-1', '2018-12-31'), 
                         data=imputed_volume_m.transpose(), columns=price_df_.columns)


In [37]:
price_df.to_csv('price_brinjal_locf.csv')
volume_df.to_csv('volume_brinjal_locf.csv')

In [99]:
df.fillna(0, inplace=True)
df = df.astype('float64')
m = df.to_numpy()
print(f"m: {m}")
m_is_nan = np.where(m == 0., True, False)
print(f"m_is_nan: {m_is_nan}")
missing_mask = np.random.rand(*m.shape) > 0.1

mask = np.where((m_is_nan | missing_mask), True, False)
print(f"mask: {mask}")

print(f"{np.size(mask) - np.sum(mask)}")

m_incomplete = m.copy()
m_incomplete[np.invert(mask)] = 0.

print(f"m_incomplete: {m_incomplete}")


m: [[   0.    0.    0. ... 1200.    0.    0.]
 [   0.    0.    0. ... 1200.    0.    0.]
 [   0.    0.    0. ... 1200.    0.    0.]
 ...
 [   0.    0.    0. ...    0. 1250. 1200.]
 [   0.    0. 1000. ...    0. 1100. 1300.]
 [   0.    0. 1000. ... 1500.  800. 1300.]]
m_is_nan: [[ True  True  True ... False  True  True]
 [ True  True  True ... False  True  True]
 [ True  True  True ... False  True  True]
 ...
 [ True  True  True ...  True False False]
 [ True  True False ...  True False False]
 [ True  True False ... False False False]]
mask: [[ True  True  True ...  True  True  True]
 [ True  True  True ...  True  True  True]
 [ True  True  True ...  True  True  True]
 ...
 [ True  True  True ...  True  True  True]
 [ True  True  True ...  True  True  True]
 [ True  True  True ...  True False False]]
95086
m_incomplete: [[   0.    0.    0. ... 1200.    0.    0.]
 [   0.    0.    0. ... 1200.    0.    0.]
 [   0.    0.    0. ... 1200.    0.    0.]
 ...
 [   0.    0.    0. ...    0. 1250.

In [64]:
# false in mask matrix indicates that the cell is the test subject

print(np.random.rand())

0.539994361987414
